# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = "../output_data/cities.csv"

weather_data = pd.read_csv(weather_data)

In [3]:
weather_data.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed
0,0,hobart,AU,1586774656,-42.88,147.33,55.40,76,75,5.82
1,1,faanui,PF,1586774935,-16.48,-151.75,82.69,72,71,10.51
2,2,eureka,US,1586774937,40.80,-124.16,45.00,81,1,3.36
3,3,cape town,ZA,1586774746,-33.93,18.42,68.00,52,40,6.93
4,4,kodiak,US,1586774908,57.79,-152.41,39.20,93,100,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure Gmaps via API key.
gmaps.configure(api_key=g_key)

locations = weather_data[["Lat", "Lng"]]

humidity = weather_data["Humidity"].astype(float)

# figure_layout = {
#     'width' : '1000px',
#     'height':'400px',
#     'border':'1px solid black',
#     'padding':'1px',
#     'margin':'0 auto 0 auto'
# }

# fig = gmaps.figure(layout=figure_layout)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

In [5]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# > Narrow down the DataFrame to find your ideal weather condition.
# ~ Example as follows:
#   ~ A max temperature lower than 80 degrees but higher than 70.
#   ~ Wind speed less than 10 mph.
#   ~ Zero cloudiness.
#   ~ Drop any rows that don't contain all three conditions. 
#   ~ You want to be sure the weather is ideal?

new_df = pd.DataFrame(weather_data, columns = ["City", "Temp", "Wind Speed", "Cloudiness"])

temp = (new_df["Temp"] <= 80) & (new_df["Temp"] > 70)
windspeed = new_df["Wind Speed"] < 10
cloudiness = new_df["Cloudiness"] == 0

new_df[temp & windspeed & cloudiness]

,City,Temp,Wind Speed,Cloudiness
69,pontalina,73.56,5.46,0
248,turayf,71.60,4.70,0
295,villarrica,73.00,5.66,0
529,guiratinga,76.73,2.82,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame(weather_data, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

In [8]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,hobart,AU,-42.88,147.33,
1,faanui,PF,-16.48,-151.75,
2,eureka,US,40.80,-124.16,
3,cape town,ZA,-33.93,18.42,
4,kodiak,US,57.79,-152.41,
...,...,...,...,...,...
581,lasa,IT,46.62,10.70,
582,bayir,TR,37.27,28.22,
583,embonas,GR,36.23,27.86,
584,vicuna,CL,-30.03,-70.71,


In [9]:
target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# Parameters Dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Request via PD
response = requests.get(base_url, params=params)

In [10]:
pprint(response.json(), depth=1)

{'html_attributions': [],
 'next_page_token': 'CrQCIQEAANpBD86tBj-k44zsWUyVKGwHXPyngfU5GOUnnfqBySaD1YXVNNPVgB5nadqrCOQYW7eMY0VaopQsVYg0DTuFwmR9Vl5rnPNdkGfjqS0iVrhIV-ZRvEH-Z3W1donPpQbVgRobtMbVoMuIGCEPGcfmdh2B5cQF_RHiXndHGKBQ8sAkuWrbh6PU0JDMbN-2U-6QUXdYPaJYYpzp6-v3QgUmGg2Kjlb7iLCzZnlUcZ3o7dN_zfJDVtQks8fjpSFfR5dH5u3G2M0Ro2NmCtUFAvHqs_LQ5aGUKSU5JIBUCPpmrO2cL8Bl9GlsFZPxJtgLGXl3CPojwADnW90DrftskE0yQaO63BEdc9CQ-PS4mzfhxs1ougcDUM6epb_oo5algmCxGvp54IjJIoUhBRjp_ZY7LsASECRkdepQJjXtvWhpAJSERK8aFIElXM3YKagW1gglRJIi5xzFW7vy',
 'results': [...],
 'status': 'OK'}


In [11]:
hotel_df.iloc[0][0]

'hobart'

In [ ]:
hotel_name = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Request & print URL
    name_address = requests.get(base_url, params=params)
    
    # Convert to JSON
    name_address = name_address.json()
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [ ]:
hotel_name

In [ ]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.to_csv('hotel_data.csv')

In [ ]:
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Hotel layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]